In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier 
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt



nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Text Mining/Dataset/summarised_weighted_discourse.csv')

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Text,oh_label,Label,user_links_dropped,spelling_corrected,hashtag_topics,...,clean_lower,tokenized_text,remaining_words,sentences,Sentiment Score (Original Text),summarised_sentences,Weighted Discourse Fragments (PDTB),Sentiment Score (PDTB split),Weighted Discourse Fragments (Dependency split),Sentiment Score (Dependency split)
0,0,149970,149988,149988,` == Clandestine industries == Hi - I note y...,0,0,` == Clandestine industries == Hi - I note you...,` == Clandestine industries == Hi - I note you...,[],...,clandestine industries hi i note you have ...,"['clandestine', 'industries', 'hi', 'i', 'note...","['clandestine', 'industries', 'hi', 'note', 'r...",['clandestine industries hi i note you have ...,0.3182,['i note you have removed the speedy deletion ...,{'i note you have removed the speedy deletion ...,0.135575,{'i note you have removed the speedy deletion ...,0.27115
1,1,136680,136697,136697,` ==Sailor Moon Musicals== I tried to add the...,0,0,` ==Sailor Moon Musicals== I tried to add the ...,` ==Sailor Moon Musicals== I tried to add the ...,[],...,sailor moon musicals i tried to add the clari...,"['sailor', 'moon', 'musicals', 'i', 'tried', '...","['sailor', 'moon', 'musicals', 'tried', 'add',...",['sailor moon musicals i tried to add the clar...,0.3612,['sailor moon musicals i tried to add the clar...,{'sailor moon musicals i tried to add the clar...,0.122480,{'sailor moon musicals i tried to add the clar...,0.00000
2,2,90772,90780,90780,:::That was my point. I wanted to rewrite the...,0,0,:::That was my point. I wanted to rewrite the ...,:::That was my point I wanted to rewrite the e...,[],...,that was my point i wanted to rewrite the enti...,"['that', 'was', 'my', 'point', 'i', 'wanted', ...","['point', 'wanted', 'rewrite', 'entire', 'plot...","['that was my point', 'i wanted to rewrite the...",0.9674,"['that was my point', 'i wanted to rewrite the...","{'that was my point': 1, 'i wanted to rewrite ...",0.168560,"{'that was my point': 1, 'i wanted to rewrite ...",0.29360
3,3,177127,177147,177147,` == Channel 4 Documentary == Nice work Prio...,0,0,` == Channel 4 Documentary == Nice work Priory...,` == Channel 4 Documentary == Nice work Priory...,[],...,channel 4 documentary nice work prioryman i...,"['channel', '4', 'documentary', 'nice', 'work'...","['channel', '4', 'documentary', 'nice', 'work'...","['channel 4 documentary nice work prioryman',...",0.7641,['channel 4 documentary nice work prioryman'...,{'channel 4 documentary nice work prioryman'...,0.271350,{'channel 4 documentary nice work prioryman'...,0.32250
4,4,122009,122025,122025,Is this species named after Sir David Attenb...,0,0,Is this species named after Sir David Attenbor...,Is this species named after Sir David Attenbor...,[],...,is this species named after sir david attenbor...,"['is', 'this', 'species', 'named', 'after', 's...","['species', 'named', 'sir', 'david', 'attenbor...",['is this species named after sir david attenb...,0.0000,['is this species named after sir david attenb...,{'is this species named after sir david attenb...,0.000000,{'is this species named after sir david attenb...,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2005,6218,6218,6218,==series scrapped on 4th july??!?== on http...,1,2,==series scrapped on 4th july??!?== on a few p...,==series scrapped on ith july??!?== on a few p...,[],...,series scrapped on 4th july on a few ppl hav l...,"['series', 'scrapped', 'on', '4th', 'july', 'o...","['series', 'scrapped', '4th', 'july', 'ppl', '...","['series scrapped on 4th july', 'on a few ppl ...",0.0000,"['series scrapped on 4th july', 'on a few ppl ...","{'series scrapped on 4th july': 1, 'on a few p...",0.000000,"{'series scrapped on 4th july': 1, 'on a few p...",0.00000
2006,2006,24195,24196,24196,"@Iloveoldtools @Angry_Feminazi No. Actually, ...",1,2,"No. Actually, she has an economics degree.",not actually she has an e

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 23 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       2010 non-null   int64  
 1   Unnamed: 0.1                                     2010 non-null   int64  
 2   Unnamed: 0.1.1                                   2010 non-null   int64  
 3   Unnamed: 0.1.1.1                                 2010 non-null   int64  
 4   Text                                             2010 non-null   object 
 5   oh_label                                         2010 non-null   int64  
 6   Label                                            2010 non-null   int64  
 7   user_links_dropped                               2010 non-null   object 
 8   spelling_corrected                               2010 non-null   object 
 9   hashtag_topics                

In [ ]:
use_df2 = df[['summarised_sentences','Sentiment Score (Dependency split)','Label']].copy()
use_df2
use_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   summarised_sentences                2010 non-null   object 
 1   Sentiment Score (Dependency split)  2010 non-null   float64
 2   Label                               2010 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 47.2+ KB


In [ ]:
df_build2 = use_df2.copy()
df_build2

,summarised_sentences,Sentiment Score (Dependency split),Label
0,['i note you have removed the speedy deletion ...,0.27115,0
1,['sailor moon musicals i tried to add the clar...,0.00000,0
2,"['that was my point', 'i wanted to rewrite the...",0.29360,0
3,['channel 4 documentary nice work prioryman'...,0.32250,0
4,['is this species named after sir david attenb...,0.00000,0
...,...,...,...
2005,"['series scrapped on 4th july', 'on a few ppl ...",0.00000,2
2006,['actually she has an economics degree'],0.00000,2
2007,['seriously i looked at your contributions the...,-0.44040,2
2008,['i love to eat rectal yoghurt'],0.63690,2


In [ ]:
y = df_build2['Label']
X = df_build2[['summarised_sentences','Sentiment Score (Dependency split)']]

In [ ]:
df_scaled_sem_dep = df_build2[['Sentiment Score (Dependency split)']]
print(scaler.fit(df_scaled_sem_dep))

MinMaxScaler()


In [ ]:
scaled_sem_dep = scaler.transform(df_scaled_sem_dep)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=460)

In [ ]:
NB_df = df_build2[['summarised_sentences','Label']].copy()

In [ ]:
NB_df['scaled_dep'] = scaled_sem_dep

In [ ]:
NB_df

,summarised_sentences,Label,scaled_dep
0,['i note you have removed the speedy deletion ...,0,0.635607
1,['sailor moon musicals i tried to add the clar...,0,0.500025
2,"['that was my point', 'i wanted to rewrite the...",0,0.646832
3,['channel 4 documentary nice work prioryman'...,0,0.661283
4,['is this species named after sir david attenb...,0,0.500025
...,...,...,...
2005,"['series scrapped on 4th july', 'on a few ppl ...",2,0.500025
2006,['actually she has an economics degree'],2,0.500025
2007,['seriously i looked at your contributions the...,2,0.279814
2008,['i love to eat rectal yoghurt'],2,0.818491


In [ ]:
y_NB = NB_df['Label']
X_NB = NB_df[['summarised_sentences','scaled_dep']]

In [ ]:
X_NB_train, X_NB_test, y_NB_train, y_NB_test = train_test_split(X_NB, y_NB, test_size=0.7, random_state=460)

In [ ]:
NB_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', NB)
                ])
NB_pipe.fit(X_NB_train,y_NB_train)

Pipeline(steps=[('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidf1', TfidfVectorizer(),
                                                  'summarised_sentences')])),
                ('classify', MultinomialNB())])

In [ ]:
# initialise model and vectorizers
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
NB = MultinomialNB()
RF = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=450)
XGB = XGBClassifier()
LGBM = lgb.LGBMClassifier()
LR = LogisticRegression(random_state=450)
vectorizer1 = TfidfVectorizer()


# construct the column transfomer
column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'summarised_sentences')],
    remainder='passthrough')

# fit the model
SVM_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', SVM)
                ])
SVM_pipe.fit(X_train,y_train)

# NB_pipe = Pipeline([
#                   ('tfidf', column_transformer),
#                   ('classify', NB)
#                 ])
# NB_pipe.fit(X_train,y_train)

RF_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', RF)
                ])
RF_pipe.fit(X_train,y_train)

XGB_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', XGB)
                ])
XGB_pipe.fit(X_train,y_train)

LGBM_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', LGBM)
                ])
LGBM_pipe.fit(X_train,y_train)

LR_pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', LR)
                ])
LR_pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidf1', TfidfVectorizer(),
                                                  'summarised_sentences')])),
                ('classify', LogisticRegression(random_state=450))])

In [ ]:
predictions_NB = NB_pipe.predict(X_NB_test)
nb_accuracy = accuracy_score(predictions_NB, y_test)*100

NB_matrix = classification_report(y_NB_test,predictions_NB,labels=[0,1,2])
print("Accuracy:", nb_accuracy)
print('Classification report : \n',NB_matrix)

Accuracy: 61.26510305614783
Classification report : 
               precision    recall  f1-score   support

           0       0.63      0.76      0.69       475
           1       0.58      0.52      0.55       463
           2       0.62      0.55      0.58       469

    accuracy                           0.61      1407
   macro avg       0.61      0.61      0.61      1407
weighted avg       0.61      0.61      0.61      1407



In [ ]:
predictions_SVM = SVM_pipe.predict(X_NB_test)
svm_accuracy = accuracy_score(predictions_SVM, y_NB_test)*100

SVM_matrix = classification_report(y_NB_test,predictions_SVM,labels=[0,1,2])
print("Accuracy:", svm_accuracy)
print('Classification report : \n',SVM_matrix)

Accuracy: 57.00071073205402
Classification report : 
               precision    recall  f1-score   support

           0       0.59      0.75      0.66       475
           1       0.75      0.19      0.31       463
           2       0.52      0.76      0.62       469

    accuracy                           0.57      1407
   macro avg       0.62      0.57      0.53      1407
weighted avg       0.62      0.57      0.53      1407



In [ ]:
predictions_LGBM = LGBM_pipe.predict(X_NB_test)

lgbm_accuracy = accuracy_score(predictions_LGBM, y_NB_test)*100
LGBM_matrix = classification_report(y_NB_test,predictions_LGBM)
print("Accuracy:", lgbm_accuracy)
print('Classification report : \n',LGBM_matrix)

Accuracy: 53.447050461975834
Classification report : 
               precision    recall  f1-score   support

           0       0.56      0.64      0.60       475
           1       0.56      0.30      0.39       463
           2       0.50      0.65      0.57       469

    accuracy                           0.53      1407
   macro avg       0.54      0.53      0.52      1407
weighted avg       0.54      0.53      0.52      1407



In [ ]:
predictions_XGB = XGB_pipe.predict(X_NB_test)

xgb_accuracy = accuracy_score(predictions_XGB, y_NB_test)*100
XGB_matrix = classification_report(y_NB_test,predictions_XGB,labels=[0,1,2])
print("Accuracy:", xgb_accuracy)
print('Classification report : \n',XGB_matrix)

Accuracy: 55.57924662402275
Classification report : 
               precision    recall  f1-score   support

           0       0.60      0.65      0.62       475
           1       0.64      0.27      0.38       463
           2       0.50      0.75      0.60       469

    accuracy                           0.56      1407
   macro avg       0.58      0.55      0.53      1407
weighted avg       0.58      0.56      0.53      1407



In [ ]:
predictions_LR = LR_pipe.predict(X_NB_test)

lr_accuracy = accuracy_score(predictions_LR, y_NB_test)*100
LR_matrix = classification_report(y_NB_test,predictions_LR)
print("Accuracy:", lr_accuracy)
print('Classification report : \n',LR_matrix)

Accuracy: 55.508173418621176
Classification report : 
               precision    recall  f1-score   support

           0       0.56      0.78      0.65       475
           1       0.87      0.12      0.21       463
           2       0.52      0.75      0.62       469

    accuracy                           0.56      1407
   macro avg       0.65      0.55      0.49      1407
weighted avg       0.65      0.56      0.49      1407

